In [1]:
import pickle
import numpy as np
import pandas as pd
import os
from tqdm.notebook import tqdm
import re

seed = 2023

In [2]:
import torch

# set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [3]:
from nlpsig_networks.scripts.swnu_network_functions import (
    swnu_network_hyperparameter_search,
    obtain_SWNUNetwork_input,
)

In [4]:
output_dir = "rumours_output"
if not os.path.isdir(output_dir):
    os.makedirs(output_dir)

## Rumours

In [5]:
%run load_sbert-embeddings.py

In [6]:
df_rumours.head()

,id,label,datetime,text,timeline_id,set
0,5.249902e+17,0,2014-10-22 18:26:23,Police have clarified that there were two shoo...,0,train
1,5.249906e+17,0,2014-10-22 18:27:58,"@CTVNews you guys ""confirmed"" there were 3 sho...",0,train
2,5.249908e+17,1,2014-10-22 18:28:46,@CTVNews get it right. http://t.co/GHYxMuzPG9,0,train
3,5.249927e+17,1,2014-10-22 18:36:29,RT @CTVNews Police have clarified that there w...,0,train
4,5.250038e+17,1,2014-10-22 19:20:41,@CTVNews @ctvsaskatoon so what happened at Rid...,0,train


In [7]:
x_data = obtain_SWNUNetwork_input(
    method="gaussian_random_projection",
    dimension=30,
    df=df_rumours,
    id_column="timeline_id",
    label_column="label",
    embeddings=sbert_embeddings,
    k=5,
    features="time_encoding",
    standardise_method=None,
    add_time_in_path=False,
)

x_data[0].shape

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/5568 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.


torch.Size([5568, 5, 415])

## SWNU Network

In [8]:
features = ["time_encoding", "timeline_index"]
standardise_method = ["z_score", None]
num_features = len(features)
add_time_in_path = True

In [9]:
num_epochs = 100
embedding_dim = 384
dimensions = [15]
swnu_hidden_dim_sizes_and_sig_depths = [([12], 3), ([10], 4)]
ffn_hidden_dim_sizes = [[64, 64], [128, 128], [256, 256], [512, 512]]
dropout_rates = [0.5, 0.1]
learning_rates = [1e-3, 1e-4, 5e-4]
seeds = [1, 12, 123]
bidirectional = True
loss = "focal"
gamma = 2
validation_metric = "f1"
patience = 5
split_indices = (
    df_rumours[df_rumours["set"] == "train"].index,
    df_rumours[df_rumours["set"] == "dev"].index,
    df_rumours[df_rumours["set"] == "test"].index,
)

In [10]:
# #dimensionality reduction
# embedding_dim = sbert_embeddings.shape[1]
# dim_reduce_method = ["umap"] #["gaussian_random_projection", "umap"]
# dimensions = [20]#[50,30,15]
# #time features
# features = "time_encoding"
# standardise_method = "standardise"
# add_time_in_path = False
# #SWNU block
# augmentation_tp = "Conv1d"
# hidden_dim_aug = None
# comb_m = "concatenation"
# log_sig = True
# conv_output_channels = [10] #[20, 10, 5]
# log_signature_dimensions_and_sig_depths = [(8, 3)]#[(30, 2), (10, 3), (6, 4)]
# bidirectional = False
# #ffn
# hidden_dim_sizes = [64]#[32,64]
# dropout_rates = [0.2]#[0.5, 0.2, 0.1]
# #overall training
# num_epochs = 100
# batch=64
# patience = 4
# learning_rates = [1e-4] #[1e-3, 1e-4, 5e-4]
# seeds = [0, 1, 12, 123, 1234]
# loss = "focal"
# gamma = 2
# validation_metric = "f1"
# split_indices = (df_rumours[df_rumours['set']=='train'].index,
#                  df_rumours[df_rumours['set']=='dev'].index,
#                  df_rumours[df_rumours['set']=='test'].index)

In [11]:
size = 35

## UMAP

In [11]:
swnu_network_umap, best_swnu_network_umap, _, __ = swnu_network_hyperparameter_search(
    num_epochs=num_epochs,
    df=df_rumours,
    id_column="timeline_id",
    label_column="label",
    embeddings=sbert_embeddings,
    y_data=y_data,
    embedding_dim=embedding_dim,
    output_dim=output_dim,
    history_lengths=[size],
    dim_reduce_methods=["umap"],
    dimensions=dimensions,
    log_signature=True,
    swnu_hidden_dim_sizes_and_sig_depths=swnu_hidden_dim_sizes_and_sig_depths,
    ffn_hidden_dim_sizes=ffn_hidden_dim_sizes,
    dropout_rates=dropout_rates,
    learning_rates=learning_rates,
    BiLSTM=bidirectional,
    seeds=seeds,
    loss=loss,
    gamma=gamma,
    device=device,
    features=features,
    standardise_method=standardise_method,
    add_time_in_path=add_time_in_path,
    split_ids=None,  # torch.tensor(df_rumours['timeline_id'].astype(int)),
    split_indices=split_indices,
    k_fold=False,
    patience=patience,
    validation_metric=validation_metric,
    results_output=f"{output_dir}/swnu_network_umap_focal_{gamma}_{size}.csv",
    verbose=False,
)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


##################################################
dimension: 15 | method: umap
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/5568 [00:00<?, ?it/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/5568 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.
saving results dataframe to CSV for this hyperparameter search in rumours_output/swnu_network_umap_focal_2_35.csv
saving the best model results dataframe to CSV for this hyperparameter search in rumours_output/swnu_network_umap_focal_2_35_best_model.csv


In [12]:
swnu_network_umap.groupby(
    ["dimensions", "swnu_hidden_dim", "ffn_hidden_dim", "dropout_rate", "learning_rate"]
).mean()

/tmp/ipykernel_1937203/2289302539.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  swnu_network_umap.groupby(["dimensions",


loss  \
dimensions swnu_hidden_dim ffn_hidden_dim dropout_rate learning_rate             
15         (10,)           (64, 64)       0.1          0.0001         0.371781   
                                                       0.0005         0.440356   
                                                       0.0010         0.360259   
                                          0.5          0.0001         0.377169   
                                                       0.0005         0.455574   
                                                       0.0010         0.348458   
                           (128, 128)     0.1          0.0001         0.524425   
                                                       0.0005         0.412525   
                                                       0.0010         0.473316   
                                          0.5          0.0001         0.350557   
                                                       0.0005         0.427026   
                                                       0.0010         0.327612   
                           (256, 256)     0.1          0.0001         0.455113   
                                                       0.0005         0.383629   
                                                       0.0010         0.402671   
                                          0.5          0.0001         0.414512   
                                                       0.0005         0.390813   
                                                       0.0010         0.432883   
                           (512, 512)     0.1          0.0001         0.559197   
                                                       0.0005         0.489585   
                                                       0.0010         0.482960   
                                          0.5          0.0001         0.438350   
                                                       0.0005         0.347589   
                                                       0.0010         0.419916   
           (12,)           (64, 64)       0.1          0.0001         0.285349   
                                                       0.0005         0.446762   
                                                       0.0010         0.343697   
                                          0.5          0.0001         0.273866   
                                                       0.0005         0.426052   
                                                       0.0010         0.480820   
                           (128, 128)     0.1          0.0001         0.319503   
                                                       0.0005         0.450899   
                                                       0.0010         0.539012   
                                          0.5          0.0001         0.307972   
                                                       0.0005         0.440530   
                                                       0.0010         0.367332   
                           (256, 256)     0.1          0.0001         0.466642   
                                                       0.0005         0.569630   
                                                       0.0010         0.601132   
                                          0.5          0.0001         0.481612   
                                                       0.0005         0.461614   
                                                       0.0010         0.497822   
                           (512, 512)     0.1          0.0001         0.385423   
                                                       0.0005         0.541610   
                                                       0.0010         0.509491   
                                          0.5          0.0001         0.379065   
                                                       0.0005         0.504722   
                                                       0

In [13]:
best_swnu_network_umap

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,seed,BiLSTM,loss_function,gamma,k_fold,n_splits,augmentation_type,hidden_dim_aug,comb_method,batch_size
0,0.389666,0.609152,0.605639,"[0.5684210526315788, 0.6428571428571428]",0.621381,"[0.6733167082294265, 0.5694444444444444]",0.614902,"[0.4918032786885246, 0.738]",0.210545,0.715302,...,1,True,focal,2,False,None,Conv1d,None,concatenation,64
0,0.549847,0.628217,0.623035,"[0.5788336933045356, 0.6672354948805461]",0.646360,"[0.7108753315649867, 0.5818452380952381]",0.635080,"[0.48816029143898, 0.782]",0.391679,0.676157,...,12,True,focal,2,False,None,Conv1d,None,concatenation,64
0,0.381554,0.643470,0.642096,"[0.6199186991869919, 0.6642728904847397]",0.651878,"[0.7011494252873564, 0.6026058631921825]",0.647778,"[0.5555555555555556, 0.74]",0.449311,0.729537,...,123,True,focal,2,False,None,Conv1d,None,concatenation,64


In [14]:
best_swnu_network_umap["f1"].mean()

0.6235898288909225

In [15]:
best_swnu_network_umap["precision"].mean()

0.6398728351356057

In [16]:
best_swnu_network_umap["recall"].mean()

0.6325865209471767

In [17]:
np.stack(best_swnu_network_umap["f1_scores"]).mean(axis=0)

array([0.58905782, 0.65812184])

In [18]:
np.stack(best_swnu_network_umap["precision_scores"]).mean(axis=0)

array([0.69511382, 0.58463185])

In [19]:
np.stack(best_swnu_network_umap["recall_scores"]).mean(axis=0)

array([0.51183971, 0.75333333])

## Unidirectional LSTM

In [12]:
(
    swnu_network_umap_uni,
    best_swnu_network_umap_uni,
    _,
    __,
) = swnu_network_hyperparameter_search(
    num_epochs=num_epochs,
    df=df_rumours,
    id_column="timeline_id",
    label_column="label",
    embeddings=sbert_embeddings,
    y_data=y_data,
    embedding_dim=embedding_dim,
    output_dim=output_dim,
    history_lengths=[size],
    dim_reduce_methods=["umap"],
    dimensions=dimensions,
    log_signature=True,
    swnu_hidden_dim_sizes_and_sig_depths=swnu_hidden_dim_sizes_and_sig_depths,
    ffn_hidden_dim_sizes=ffn_hidden_dim_sizes,
    dropout_rates=dropout_rates,
    learning_rates=learning_rates,
    BiLSTM=False,
    seeds=seeds,
    loss=loss,
    gamma=gamma,
    device=device,
    features=features,
    standardise_method=standardise_method,
    add_time_in_path=add_time_in_path,
    split_ids=None,  # torch.tensor(df_rumours['timeline_id'].astype(int)),
    split_indices=split_indices,
    k_fold=False,
    patience=patience,
    validation_metric=validation_metric,
    results_output=f"{output_dir}/swnu_network_umap_focal_{gamma}_{size}_uni.csv",
    verbose=False,
)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


##################################################
dimension: 15 | method: umap
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/5568 [00:00<?, ?it/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/5568 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.
saving results dataframe to CSV for this hyperparameter search in rumours_output/swnu_network_umap_focal_2_35_uni.csv
saving the best model results dataframe to CSV for this hyperparameter search in rumours_output/swnu_network_umap_focal_2_35_uni_best_model.csv


In [13]:
swnu_network_umap_uni.groupby(
    ["dimensions", "swnu_hidden_dim", "ffn_hidden_dim", "dropout_rate", "learning_rate"]
).mean()

/tmp/ipykernel_2865884/4035633852.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  swnu_network_umap_uni.groupby(["dimensions",


loss  \
dimensions swnu_hidden_dim ffn_hidden_dim dropout_rate learning_rate             
15         (10,)           (64, 64)       0.1          0.0001         0.401949   
                                                       0.0005         0.370616   
                                                       0.0010         0.347093   
                                          0.5          0.0001         0.306184   
                                                       0.0005         0.503728   
                                                       0.0010         0.315259   
                           (128, 128)     0.1          0.0001         0.356136   
                                                       0.0005         0.508568   
                                                       0.0010         0.360384   
                                          0.5          0.0001         0.314818   
                                                       0.0005         0.503755   
                                                       0.0010         0.456130   
                           (256, 256)     0.1          0.0001         0.331225   
                                                       0.0005         0.400315   
                                                       0.0010         0.381459   
                                          0.5          0.0001         0.487248   
                                                       0.0005         0.453355   
                                                       0.0010         0.357499   
                           (512, 512)     0.1          0.0001         0.407519   
                                                       0.0005         0.769205   
                                                       0.0010         0.505887   
                                          0.5          0.0001         0.416141   
                                                       0.0005         0.402505   
                                                       0.0010         0.311081   
           (12,)           (64, 64)       0.1          0.0001         0.310053   
                                                       0.0005         0.357353   
                                                       0.0010         0.478149   
                                          0.5          0.0001         0.269855   
                                                       0.0005         0.475538   
                                                       0.0010         0.416742   
                           (128, 128)     0.1          0.0001         0.392682   
                                                       0.0005         0.528559   
                                                       0.0010         0.509632   
                                          0.5          0.0001         0.267770   
                                                       0.0005         0.453261   
                                                       0.0010         0.484720   
                           (256, 256)     0.1          0.0001         0.349295   
                                                       0.0005         0.433796   
                                                       0.0010         0.376109   
                                          0.5          0.0001         0.334049   
                                                       0.0005         0.502830   
                                                       0.0010         0.467905   
                           (512, 512)     0.1          0.0001         0.493946   
                                                       0.0005         0.433638   
                                                       0.0010         0.491985   
                                          0.5          0.0001         0.345170   
                                                       0.0005         0.477446   
                                                       0

In [14]:
best_swnu_network_umap_uni

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,seed,BiLSTM,loss_function,gamma,k_fold,n_splits,augmentation_type,hidden_dim_aug,comb_method,batch_size
0,0.447465,0.615825,0.598484,"[0.5150421179302046, 0.6819258089976322]",0.661049,"[0.7588652482269503, 0.5632333767926988]",0.626900,"[0.38979963570127507, 0.864]",0.201085,0.718861,...,1,False,focal,2,False,None,Conv1d,None,concatenation,64
0,0.913771,0.587226,0.573225,"[0.49592549476135034, 0.6505246166263116]",0.616214,"[0.6870967741935484, 0.5453315290933695]",0.596989,"[0.3879781420765027, 0.806]",0.333290,0.722420,...,12,False,focal,2,False,None,Conv1d,None,concatenation,64
0,0.946379,0.604385,0.595901,"[0.5373467112597548, 0.6544546211490425]",0.626578,"[0.6925287356321839, 0.5606276747503567]",0.612490,"[0.43897996357012753, 0.786]",0.279069,0.729537,...,123,False,focal,2,False,None,Conv1d,None,concatenation,64


In [15]:
best_swnu_network_umap_uni["f1"].mean()

0.5892032284540494

In [16]:
best_swnu_network_umap_uni["precision"].mean()

0.6346138897815179

In [17]:
best_swnu_network_umap_uni["recall"].mean()

0.6121262902246508

In [18]:
np.stack(best_swnu_network_umap_uni["f1_scores"]).mean(axis=0)

array([0.51610477, 0.66230168])

In [19]:
np.stack(best_swnu_network_umap_uni["precision_scores"]).mean(axis=0)

array([0.71283025, 0.55639753])

In [20]:
np.stack(best_swnu_network_umap_uni["recall_scores"]).mean(axis=0)

array([0.40558591, 0.81866667])

## GRP

In [20]:
swnu_network_grp, best_swnu_network_grp, _, __ = swnu_network_hyperparameter_search(
    num_epochs=num_epochs,
    df=df_rumours,
    id_column="timeline_id",
    label_column="label",
    embeddings=sbert_embeddings,
    y_data=y_data,
    embedding_dim=embedding_dim,
    output_dim=output_dim,
    history_lengths=[size],
    dim_reduce_methods=["gaussian_random_projection"],
    dimensions=dimensions,
    log_signature=True,
    swnu_hidden_dim_sizes_and_sig_depths=swnu_hidden_dim_sizes_and_sig_depths,
    ffn_hidden_dim_sizes=ffn_hidden_dim_sizes,
    dropout_rates=dropout_rates,
    learning_rates=learning_rates,
    BiLSTM=bidirectional,
    seeds=seeds,
    loss=loss,
    gamma=gamma,
    device=device,
    features=features,
    standardise_method=standardise_method,
    add_time_in_path=add_time_in_path,
    split_ids=None,  # torch.tensor(df_rumours['timeline_id'].astype(int)),
    split_indices=split_indices,
    k_fold=False,
    patience=patience,
    validation_metric=validation_metric,
    results_output=f"{output_dir}/swnu_network_grp_focal_{gamma}_{size}.csv",
    verbose=False,
)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


##################################################
dimension: 15 | method: gaussian_random_projection
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/5568 [00:00<?, ?it/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/5568 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.
saving results dataframe to CSV for this hyperparameter search in rumours_output/swnu_network_grp_focal_2_35.csv
saving the best model results dataframe to CSV for this hyperparameter search in rumours_output/swnu_network_grp_focal_2_35_best_model.csv


In [21]:
swnu_network_grp.groupby(
    ["dimensions", "swnu_hidden_dim", "ffn_hidden_dim", "dropout_rate", "learning_rate"]
).mean()

/tmp/ipykernel_1937203/1369330714.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  swnu_network_grp.groupby(["dimensions",


loss  \
dimensions swnu_hidden_dim ffn_hidden_dim dropout_rate learning_rate             
15         (10,)           (64, 64)       0.1          0.0001         0.288617   
                                                       0.0005         0.492734   
                                                       0.0010         0.439261   
                                          0.5          0.0001         0.360769   
                                                       0.0005         0.451771   
                                                       0.0010         0.333791   
                           (128, 128)     0.1          0.0001         0.347467   
                                                       0.0005         0.404913   
                                                       0.0010         0.934171   
                                          0.5          0.0001         0.376253   
                                                       0.0005         0.548737   
                                                       0.0010         0.403334   
                           (256, 256)     0.1          0.0001         0.432431   
                                                       0.0005         0.404785   
                                                       0.0010         0.438982   
                                          0.5          0.0001         0.399132   
                                                       0.0005         0.499557   
                                                       0.0010         0.385910   
                           (512, 512)     0.1          0.0001         0.533242   
                                                       0.0005         0.490148   
                                                       0.0010         0.563264   
                                          0.5          0.0001         0.358362   
                                                       0.0005         0.413423   
                                                       0.0010         0.429543   
           (12,)           (64, 64)       0.1          0.0001         0.369212   
                                                       0.0005         0.405538   
                                                       0.0010         0.666847   
                                          0.5          0.0001         0.361396   
                                                       0.0005         0.490110   
                                                       0.0010         0.383952   
                           (128, 128)     0.1          0.0001         0.363327   
                                                       0.0005         0.455098   
                                                       0.0010         0.612938   
                                          0.5          0.0001         0.399248   
                                                       0.0005         0.432102   
                                                       0.0010         0.462414   
                           (256, 256)     0.1          0.0001         0.468246   
                                                       0.0005         0.573220   
                                                       0.0010         0.525558   
                                          0.5          0.0001         0.351565   
                                                       0.0005         0.476111   
                                                       0.0010         0.454877   
                           (512, 512)     0.1          0.0001         0.369756   
                                                       0.0005         0.469823   
                                                       0.0010         0.519292   
                                          0.5          0.0001         0.372588   
                                                       0.0005         0.442898   
                                                       0

In [22]:
best_swnu_network_grp

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,seed,BiLSTM,loss_function,gamma,k_fold,n_splits,augmentation_type,hidden_dim_aug,comb_method,batch_size
0,0.368646,0.635844,0.634440,"[0.6117886178861788, 0.6570915619389588]",0.644023,"[0.6919540229885057, 0.5960912052117264]",0.640135,"[0.5482695810564663, 0.732]",0.187047,0.779359,...,1,True,focal,2,False,None,Conv1d,None,concatenation,64
0,0.429516,0.608198,0.607833,"[0.6197964847363553, 0.5958702064896755]",0.607886,"[0.6296992481203008, 0.586073500967118]",0.608100,"[0.6102003642987249, 0.606]",0.194422,0.750890,...,12,True,focal,2,False,None,Conv1d,None,concatenation,64
0,0.700507,0.582460,0.580948,"[0.5557809330628802, 0.6061151079136691]",0.588828,"[0.6270022883295194, 0.5506535947712419]",0.586545,"[0.4990892531876138, 0.674]",0.194164,0.740214,...,123,True,focal,2,False,None,Conv1d,None,concatenation,64


In [23]:
best_swnu_network_grp["f1"].mean()

0.607740485337953

In [24]:
best_swnu_network_grp["precision"].mean()

0.6135789767314019

In [25]:
best_swnu_network_grp["recall"].mean()

0.6115931997571341

In [26]:
np.stack(best_swnu_network_grp["f1_scores"]).mean(axis=0)

array([0.59578868, 0.61969229])

In [27]:
np.stack(best_swnu_network_grp["precision_scores"]).mean(axis=0)

array([0.64955185, 0.5776061 ])

In [28]:
np.stack(best_swnu_network_grp["recall_scores"]).mean(axis=0)

array([0.55251973, 0.67066667])

## Unidirectional LSTM

In [21]:
(
    swnu_network_grp_uni,
    best_swnu_network_grp_uni,
    _,
    __,
) = swnu_network_hyperparameter_search(
    num_epochs=num_epochs,
    df=df_rumours,
    id_column="timeline_id",
    label_column="label",
    embeddings=sbert_embeddings,
    y_data=y_data,
    embedding_dim=embedding_dim,
    output_dim=output_dim,
    history_lengths=[size],
    dim_reduce_methods=["gaussian_random_projection"],
    dimensions=dimensions,
    log_signature=True,
    swnu_hidden_dim_sizes_and_sig_depths=swnu_hidden_dim_sizes_and_sig_depths,
    ffn_hidden_dim_sizes=ffn_hidden_dim_sizes,
    dropout_rates=dropout_rates,
    learning_rates=learning_rates,
    BiLSTM=False,
    seeds=seeds,
    loss=loss,
    gamma=gamma,
    device=device,
    features=features,
    standardise_method=standardise_method,
    add_time_in_path=add_time_in_path,
    split_ids=None,  # torch.tensor(df_rumours['timeline_id'].astype(int)),
    split_indices=split_indices,
    k_fold=False,
    patience=patience,
    validation_metric=validation_metric,
    results_output=f"{output_dir}/swnu_network_grp_focal_{gamma}_{size}_uni.csv",
    verbose=False,
)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


##################################################
dimension: 15 | method: gaussian_random_projection
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/5568 [00:00<?, ?it/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/5568 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.
saving results dataframe to CSV for this hyperparameter search in rumours_output/swnu_network_grp_focal_2_35_uni.csv
saving the best model results dataframe to CSV for this hyperparameter search in rumours_output/swnu_network_grp_focal_2_35_uni_best_model.csv


In [22]:
swnu_network_grp_uni.groupby(
    ["dimensions", "swnu_hidden_dim", "ffn_hidden_dim", "dropout_rate", "learning_rate"]
).mean()

/tmp/ipykernel_2865884/2827013940.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  swnu_network_grp_uni.groupby(["dimensions",


loss  \
dimensions swnu_hidden_dim ffn_hidden_dim dropout_rate learning_rate             
15         (10,)           (64, 64)       0.1          0.0001         0.339255   
                                                       0.0005         0.453027   
                                                       0.0010         0.624346   
                                          0.5          0.0001         0.322910   
                                                       0.0005         0.549827   
                                                       0.0010         0.455007   
                           (128, 128)     0.1          0.0001         0.335961   
                                                       0.0005         0.456644   
                                                       0.0010         0.589034   
                                          0.5          0.0001         0.423346   
                                                       0.0005         0.555672   
                                                       0.0010         0.476452   
                           (256, 256)     0.1          0.0001         0.450159   
                                                       0.0005         0.611294   
                                                       0.0010         0.610260   
                                          0.5          0.0001         0.349612   
                                                       0.0005         0.564097   
                                                       0.0010         0.386834   
                           (512, 512)     0.1          0.0001         0.388409   
                                                       0.0005         0.495732   
                                                       0.0010         0.387514   
                                          0.5          0.0001         0.381780   
                                                       0.0005         0.491186   
                                                       0.0010         0.475687   
           (12,)           (64, 64)       0.1          0.0001         0.369444   
                                                       0.0005         0.476139   
                                                       0.0010         0.418035   
                                          0.5          0.0001         0.297237   
                                                       0.0005         0.389072   
                                                       0.0010         0.389836   
                           (128, 128)     0.1          0.0001         0.432063   
                                                       0.0005         0.470221   
                                                       0.0010         0.418157   
                                          0.5          0.0001         0.257963   
                                                       0.0005         0.429027   
                                                       0.0010         0.544181   
                           (256, 256)     0.1          0.0001         0.424406   
                                                       0.0005         0.407069   
                                                       0.0010         0.456825   
                                          0.5          0.0001         0.310308   
                                                       0.0005         0.454589   
                                                       0.0010         0.547538   
                           (512, 512)     0.1          0.0001         0.423968   
                                                       0.0005         0.474828   
                                                       0.0010         0.477870   
                                          0.5          0.0001         0.406978   
                                                       0.0005         0.525150   
                                                       0

In [23]:
best_swnu_network_grp_uni

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,seed,BiLSTM,loss_function,gamma,k_fold,n_splits,augmentation_type,hidden_dim_aug,comb_method,batch_size
0,0.535631,0.599619,0.586766,"[0.5138888888888888, 0.6596434359805511]",0.629629,"[0.7047619047619048, 0.5544959128065395]",0.609186,"[0.40437158469945356, 0.814]",0.173444,0.765125,...,1,False,focal,2,False,None,Conv1d,None,concatenation,64
0,0.746817,0.560534,0.559091,"[0.5338725985844287, 0.5843101893597836]",0.566010,"[0.6, 0.5320197044334976]",0.564437,"[0.4808743169398907, 0.648]",0.320938,0.715302,...,12,False,focal,2,False,None,Conv1d,None,concatenation,64
0,0.409842,0.586273,0.584960,"[0.5616161616161616, 0.6083032490974729]",0.592331,"[0.6303854875283447, 0.5542763157894737]",0.590188,"[0.5063752276867031, 0.674]",0.178942,0.782918,...,123,False,focal,2,False,None,Conv1d,None,concatenation,64


In [24]:
best_swnu_network_grp_uni["f1"].mean()

0.5769390872545478

In [25]:
best_swnu_network_grp_uni["precision"].mean()

0.5959898875532933

In [26]:
best_swnu_network_grp_uni["recall"].mean()

0.5879368548876746

In [27]:
np.stack(best_swnu_network_grp_uni["f1_scores"]).mean(axis=0)

array([0.53645922, 0.61741896])

In [28]:
np.stack(best_swnu_network_grp_uni["precision_scores"]).mean(axis=0)

array([0.64504913, 0.54693064])

In [29]:
np.stack(best_swnu_network_grp_uni["recall_scores"]).mean(axis=0)

array([0.46387371, 0.712     ])